In [1]:
%matplotlib inline

import os
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
import time
from django.contrib.gis.geos import Point, fromstr, GEOSGeometry
from datetime import datetime, timedelta
import vaex
import vaex.ml
from joblib import delayed, Parallel, load, parallel_backend
# from haversine import haversine, Unit

dir = Path.cwd()
outdir = os.path.join(dir, 'output')
gtfs_records_zip = os.path.join(dir, 'data', 'GtfsRRecords.zip')
gtfs_csv_zip = os.path.join(outdir, 'gtfsr_csv.zip')
gtfs_final_csv_path = os.path.join(outdir, 'gtfsr.csv')
gtfs_processed_csv_path = os.path.join(outdir, "gtfsr_processed.csv")
scats = os.path.join(dir, 'output', 'scats_model.json')

In [9]:
df = vaex.from_csv(gtfs_processed_csv_path, convert=True)
df

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,lat,lon,direction_angle,shape_dist_between,init_time,hour,dow,p_avg_vol
0,11777.2.60-13-b12-1.22.I,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,8230DB007229,15:00:00,15:00:00,0.0,53.3300889307077,-6.45108278609524,63.09549602776408,0.0,15:00:00,15,6,167.05279073328407
1,11777.2.60-13-b12-1.22.I,20210116,18:00:00,1,0,0,2021-01-16 18:50:57,8230DB007229,15:00:00,15:00:00,0.0,53.3300889307077,-6.45108278609524,63.09549602776408,0.0,15:00:00,15,6,167.05279073328407
2,11777.2.60-13-b12-1.22.I,20210123,14:00:00,1,0,0,2021-01-23 13:31:16,8230DB007229,15:00:00,15:00:00,0.0,53.3300889307077,-6.45108278609524,63.09549602776408,0.0,15:00:00,15,5,177.39385447375147
3,11777.2.60-13-b12-1.22.I,20210130,14:00:00,1,0,0,2021-01-30 13:31:03,8230DB007229,15:00:00,15:00:00,0.0,53.3300889307077,-6.45108278609524,63.09549602776408,0.0,15:00:00,15,5,177.39385447375147
4,11379.2.60-13-b12-1.19.O,20210109,18:00:00,1,0,0,2021-01-09 19:28:02,8240DB000324,15:00:00,15:00:00,0.0,53.4177226807655,-6.27864416912571,-116.93979484783767,0.0,15:00:00,15,6,62.97226861960824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"1,164,416",12640.3.60-130-b12-1.74.I,20210131,23:30:00,18,1200,1200,2021-01-31 23:59:45,8220DB001729,00:38:34,00:38:34,4808.93,53.3584969058401,-6.19031550974047,-107.23111263684194,304.8600000000006,00:30:00,0,0,12.628966054289007
"1,164,417",12877.3.60-44-b12-1.246.O,20210131,23:00:00,68,300,300,2021-01-31 23:58:50,8250DB003478,00:52:11,00:52:11,22858.06,53.2390505484196,-6.19616830395543,155.4154238326518,487.3900000000031,00:00:00,0,0,30.316506594416804
"1,164,418",12750.3.60-40-b12-1.206.O,20210131,23:00:00,79,120,120,2021-01-31 23:59:45,8230DB004690,00:57:43,00:57:43,28066.88,53.3495094392524,-6.40519639715847,-118.90642649102342,264.3100000000013,00:00:00,0,0,18.25027855067062
"1,164,419",12355.3.60-37-b12-1.39.O,20210131,23:20:00,56,120,120,2021-01-31 23:59:45,8240DB007227,00:56:18,00:56:18,19188.9,53.3837882586066,-6.4031036640899,-67.67425709043938,240.34000000000012,00:20:00,0,0,17.70530015090147


In [7]:
len(df)

873

In [5]:
df.describe().T

,dtype,count,NA,mean,std,min,max
trip_id,str,873,0,--,--,--,--
start_date,int64,873,0,20210118.625429552,2.487469,20210109,20210122
start_time,str,873,0,--,--,--,--
stop_sequence,int64,873,0,31.097365406643757,22.856009,1,91
departure,int64,873,0,105.91065292096219,197.748788,-420,900
arrival,int64,873,0,103.16151202749141,197.471051,-420,900
timestamp,str,873,0,--,--,--,--
stop_id,str,873,0,--,--,--,--
arrival_time,str,873,0,--,--,--,--
departure_time,str,873,0,--,--,--,--


In [6]:
df[(df['hour'] == 0) & (df['dow'] == 0)]

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,lat,lon,direction_angle,shape_dist_between,hour,dow,p_avg_vol
,,,,,,,,,,,,,,,,,,


In [24]:
df.plot_widget(df.lon, df.lat, shape=512, colormap='plasma', f='log1p', limits='minmax')

Heatmap(children=[ToolsToolbar(interact_value=None, supports_normalize=False, template='<template>\n  <v-toolb…